In [ ]:
import geopandas as gpd
import pandas as pd
from ipyleaflet import Map, GeoJSON
import ipywidgets as widgets

# 1. Weltkarte laden
world = gpd.read_file("./data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

# 2. CSV laden (ersetze durch deine tatsächlichen Pfade & Encoding)
df = pd.read_csv("./data/assets/Co2_Stats.csv", sep='\t', encoding='utf-16')
df = df.rename(columns={'Country': 'ADMIN'})

# Optional: Länder-Namen angleichen
country_name_map = {
    "Czechia": "Czech Republic",
    "Slovakia": "Slovak Republic",
    "Bosnia and Herzegovina": "Bosnia and Herz.",
    "North Macedonia": "Macedonia",
    "Russia": "Russian Federation"
}
df['ADMIN'] = df['ADMIN'].replace(country_name_map)

# 3. Nur eindeutige Länder aus CSV extrahieren
selected_countries = df['ADMIN'].unique().tolist()

# 4. Länder aus Shapefile herausfiltern
selected_shapes = world[world['ADMIN'].isin(selected_countries)]

# 5. GeoJSON mit einfärben
geo = GeoJSON(
    data=selected_shapes.__geo_interface__,
    style={
        'color': 'black',
        'weight': 1,
        'fillColor': 'orange',
        'fillOpacity': 0.6
    },
    name='Länder aus CSV'
)

# 6. Karte erzeugen
m = Map(center=[54.0, 15.0], zoom=4)
m.add_layer(geo)
m


/var/folders/7g/n_c_1w8j6nq8cmbcbnl7mpcc0000gn/T/ipykernel_53565/4238406504.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  europe_emissions = europe.merge(emissions_df, how='left', on='ADMIN').fillna(0)


Map(center=[54.0, 15.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [10]:
import geopandas as gpd

# Shapefile laden
world = gpd.read_file("./data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")
europe = world[world['CONTINENT'] == 'Europe']

# Englische Ländernamen ausgeben (Spalte 'ADMIN')
geo_names = sorted(europe['ADMIN'].unique())
print("Englische Ländernamen aus der Geodaten-Spalte 'ADMIN':")
for name in geo_names:
    print(name)
    

Englische Ländernamen aus der Geodaten-Spalte 'ADMIN':
Albania
Austria
Belarus
Belgium
Bosnia and Herzegovina
Bulgaria
Croatia
Czechia
Denmark
Estonia
Finland
France
Germany
Greece
Hungary
Iceland
Ireland
Italy
Kosovo
Latvia
Lithuania
Luxembourg
Moldova
Montenegro
Netherlands
North Macedonia
Norway
Poland
Portugal
Republic of Serbia
Romania
Russia
Slovakia
Slovenia
Spain
Sweden
Switzerland
Ukraine
United Kingdom


In [14]:
%pip install pycountry
%pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 250 kB 3.1 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [24]:
import pandas as pd
import plotly.graph_objects as go

# Daten vorbereiten (wie gehabt)
df = pd.read_excel("./data/assets/Co2_Stats.xlsx")

if df['t CO2 equivalent'].dtype == object:
    df['t CO2 equivalent'] = df['t CO2 equivalent'].str.replace(',', '').astype(float)

def jahr_gruppe(jahr):
    if jahr in [2018, 2019]:
        return '2018-2019'
    elif jahr in [2020, 2021]:
        return '2020-2021'
    elif jahr in [2022, 2023]:
        return '2022-2023'
    else:
        return 'Andere'

df['Jahr Gruppe'] = df['Jahr von Date'].apply(jahr_gruppe)

df_grouped = df.groupby(['Country', 'Jahr Gruppe'], as_index=False)['t CO2 equivalent'].sum()

# Für jede Jahr-Gruppe eigene Daten vorbereiten
groups = df_grouped['Jahr Gruppe'].unique()

data = []
for group in groups:
    dff = df_grouped[df_grouped['Jahr Gruppe'] == group]
    choropleth = go.Choropleth(
        locations=dff['Country'],
        locationmode='country names',
        z=dff['t CO2 equivalent'],
        colorscale='Viridis',
        colorbar_title="Emissionen (t CO2 eq.)",
        visible=(group == groups[0])  # nur erste Gruppe sichtbar
    )
    data.append(choropleth)

# Buttons fürs Dropdown erstellen
buttons = []
for i, group in enumerate(groups):
    args_visibility = [False] * len(groups)
    args_visibility[i] = True
    button = dict(
        label=group,
        method="update",
        args=[{"visible": args_visibility},
              {"title": f"CO2 Emissionen: {group}"}]
    )
    buttons.append(button)

# Layout mit Dropdown Menü
layout = go.Layout(
    title=f"CO2 Emissionen: {groups[0]}",
    geo=dict(showframe=False, showcoastlines=True),
    width=1000,
    height=700,
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=0.1,
        y=1.1,
        xanchor='left',
        yanchor='top'
    )]
)


fig = go.Figure(data=data, layout=layout)
fig.show()
